In [1]:
# PySpark SQL = Structured APIs in Spark
#     DataFrame API (Python objects, typed operations)
#     SQL API (spark.sql("SELECT ..."))
# Works on tabular data: rows + columns + schema.
# Optimized by Catalyst optimizer and Tungsten execution engine.
# Common use cases:
#     ETL (Extract–Transform–Load)
#     Reporting and dashboards
#     Data warehouse style queries
#     Joining multiple datasets

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").appName("demo").getOrCreate()

In [3]:
emp_df.show()

+------+-----+-------+-------+-------+
|emp_id| name|   dept| salary|dept_id|
+------+-----+-------+-------+-------+
|     1| Ravi|  Sales|50000.0|     10|
|     2|Priya|     HR|60000.0|     20|
|     3| John|  Sales|45000.0|     10|
|     4|Meera|Finance|70000.0|     30|
|     5|Rahul|   NULL|55000.0|   NULL|
+------+-----+-------+-------+-------+



In [2]:
from pyspark.sql.types import *
emp_data = [
    (1, "Ravi",   "Sales",   50000.0, 10),
    (2, "Priya",  "HR",      60000.0, 20),
    (3, "John",   "Sales",   45000.0, 10),
    (4, "Meera",  "Finance", 70000.0, 30),
    (5, "Rahul",  None,      55000.0, None)   # dept may be null
]

emp_schema = StructType([
    StructField("emp_id", IntegerType(), False),
    StructField("name",   StringType(),  True),
    StructField("dept",   StringType(),  True),
    StructField("salary", DoubleType(),  True),
    StructField("dept_id",IntegerType(), True)
])

emp_df = spark.createDataFrame(emp_data, schema=emp_schema)
emp_df.show()


+------+-----+-------+-------+-------+
|emp_id| name|   dept| salary|dept_id|
+------+-----+-------+-------+-------+
|     1| Ravi|  Sales|50000.0|     10|
|     2|Priya|     HR|60000.0|     20|
|     3| John|  Sales|45000.0|     10|
|     4|Meera|Finance|70000.0|     30|
|     5|Rahul|   NULL|55000.0|   NULL|
+------+-----+-------+-------+-------+



In [4]:
dept_data = [
    (10, "Sales",   "Chennai"),
    (20, "HR",      "Bangalore"),
    (30, "Finance", "Mumbai"),
    (40, "IT",      "Delhi")
]

dept_schema = StructType([
    StructField("dept_id", IntegerType(), False),
    StructField("dept_name", StringType(), True),
    StructField("location", StringType(), True)
])

dept_df = spark.createDataFrame(dept_data, schema=dept_schema)
dept_df.show()


+-------+---------+---------+
|dept_id|dept_name| location|
+-------+---------+---------+
|     10|    Sales|  Chennai|
|     20|       HR|Bangalore|
|     30|  Finance|   Mumbai|
|     40|       IT|    Delhi|
+-------+---------+---------+



In [5]:
emp_df.createOrReplaceTempView("emp")
dept_df.createOrReplaceTempView("dept")


In [6]:
spark.catalog.listTables()

[Table(name='dept', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='emp', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True)]

In [7]:
emp_df.select("emp_id","name","salary").show()

+------+-----+-------+
|emp_id| name| salary|
+------+-----+-------+
|     1| Ravi|50000.0|
|     2|Priya|60000.0|
|     3| John|45000.0|
|     4|Meera|70000.0|
|     5|Rahul|55000.0|
+------+-----+-------+



In [9]:
spark.sql("select emp_id,name,salary from emp").show()


+------+-----+-------+
|emp_id| name| salary|
+------+-----+-------+
|     1| Ravi|50000.0|
|     2|Priya|60000.0|
|     3| John|45000.0|
|     4|Meera|70000.0|
|     5|Rahul|55000.0|
+------+-----+-------+



In [13]:

from pyspark.sql.functions import col
emp_df.filter( col("salary") > 30000).select("emp_id","name","salary").show()

+------+-----+-------+
|emp_id| name| salary|
+------+-----+-------+
|     1| Ravi|50000.0|
|     2|Priya|60000.0|
|     3| John|45000.0|
|     4|Meera|70000.0|
|     5|Rahul|55000.0|
+------+-----+-------+



In [14]:
spark.sql("""
           select emp_id,name,salary
           from emp
           where salary >=50000 """).show()

+------+-----+-------+
|emp_id| name| salary|
+------+-----+-------+
|     1| Ravi|50000.0|
|     2|Priya|60000.0|
|     4|Meera|70000.0|
|     5|Rahul|55000.0|
+------+-----+-------+



In [15]:
spark.sql("""
            select emp_id,name,salary,dept_id
            from emp
            where dept_id in (10,20) and salary >40000 """).show()
            

+------+-----+-------+-------+
|emp_id| name| salary|dept_id|
+------+-----+-------+-------+
|     1| Ravi|50000.0|     10|
|     2|Priya|60000.0|     20|
|     3| John|45000.0|     10|
+------+-----+-------+-------+



In [16]:
spark.sql(""" select 
                emp_id employee_id ,
                name employee_name,
                salary ,
                dept_id department_id
            from emp
            order by salary desc """).show()

+-----------+-------------+-------+-------------+
|employee_id|employee_name| salary|department_id|
+-----------+-------------+-------+-------------+
|          4|        Meera|70000.0|           30|
|          2|        Priya|60000.0|           20|
|          5|        Rahul|55000.0|         NULL|
|          1|         Ravi|50000.0|           10|
|          3|         John|45000.0|           10|
+-----------+-------------+-------+-------------+



In [17]:
spark.sql(""" select 
                emp_id employee_id ,
                name employee_name,
                salary ,
                dept_id department_id
            from emp
            order by dept, salary desc """).show()

+-----------+-------------+-------+-------------+
|employee_id|employee_name| salary|department_id|
+-----------+-------------+-------+-------------+
|          5|        Rahul|55000.0|         NULL|
|          4|        Meera|70000.0|           30|
|          2|        Priya|60000.0|           20|
|          1|         Ravi|50000.0|           10|
|          3|         John|45000.0|           10|
+-----------+-------------+-------+-------------+



In [18]:
spark.sql(""" select 
                emp_id employee_id ,
                name employee_name,
                salary ,
                d.dept_id department_id,
                dept_name department_name,
                location
            from emp e left join dept d
            on e.dept_id =d.dept_id
            order by d.dept_id """).show()

+-----------+-------------+-------+-------------+---------------+---------+
|employee_id|employee_name| salary|department_id|department_name| location|
+-----------+-------------+-------+-------------+---------------+---------+
|          5|        Rahul|55000.0|         NULL|           NULL|     NULL|
|          3|         John|45000.0|           10|          Sales|  Chennai|
|          1|         Ravi|50000.0|           10|          Sales|  Chennai|
|          2|        Priya|60000.0|           20|             HR|Bangalore|
|          4|        Meera|70000.0|           30|        Finance|   Mumbai|
+-----------+-------------+-------+-------------+---------------+---------+



In [33]:
from pyspark.sql.functions import *
emp_df.groupBy("dept").agg(sum(col("salary")).alias("total_salary")).filter(col("total_salary") > 70000).show()

+-----+------------+
| dept|total_salary|
+-----+------------+
|Sales|     95000.0|
+-----+------------+



In [34]:
spark.sql("""
             select dept,
                     sum(salary) as total_sal,
                     max(salary) as max_sal,
                     min(salary) as min_sal,
                     avg(salary) as avg_sal,
                     count(salary) as no_of_sal
            from emp 
               group by dept 
               having sum(salary) > 70000
               """).show()

+-----+---------+-------+-------+-------+---------+
| dept|total_sal|max_sal|min_sal|avg_sal|no_of_sal|
+-----+---------+-------+-------+-------+---------+
|Sales|  95000.0|50000.0|45000.0|47500.0|        2|
+-----+---------+-------+-------+-------+---------+



In [19]:
spark.sql("""
             select dept,
                     sum(salary) as total_sal,
                     max(salary) as max_sal,
                     min(salary) as min_sal,
                     avg(salary) as avg_sal,
                     count(salary) as no_of_sal
            from emp 
               group by dept """).show()

+-------+---------+-------+-------+-------+---------+
|   dept|total_sal|max_sal|min_sal|avg_sal|no_of_sal|
+-------+---------+-------+-------+-------+---------+
|  Sales|  95000.0|50000.0|45000.0|47500.0|        2|
|     HR|  60000.0|60000.0|60000.0|60000.0|        1|
|Finance|  70000.0|70000.0|70000.0|70000.0|        1|
|   NULL|  55000.0|55000.0|55000.0|55000.0|        1|
+-------+---------+-------+-------+-------+---------+



In [35]:
spark.sql("""
           select name,
                  upper(name) ename_upper,
                  lower(name) ename_lower,
                  initcap(name) ename_initcap,
                  substring(name,1,3) ename_sub,
                  concat(emp_id,'|',name) id_name,
                  concat_ws(' ',emp_id,name,salary) id_name_sal
            from  emp """).show()


+-----+-----------+-----------+-------------+---------+-------+---------------+
| name|ename_upper|ename_lower|ename_initcap|ename_sub|id_name|    id_name_sal|
+-----+-----------+-----------+-------------+---------+-------+---------------+
| Ravi|       RAVI|       ravi|         Ravi|      Rav| 1|Ravi| 1 Ravi 50000.0|
|Priya|      PRIYA|      priya|        Priya|      Pri|2|Priya|2 Priya 60000.0|
| John|       JOHN|       john|         John|      Joh| 3|John| 3 John 45000.0|
|Meera|      MEERA|      meera|        Meera|      Mee|4|Meera|4 Meera 70000.0|
|Rahul|      RAHUL|      rahul|        Rahul|      Rah|5|Rahul|5 Rahul 55000.0|
+-----+-----------+-----------+-------------+---------+-------+---------------+



In [37]:
df = spark.range(1)
df.createOrReplaceTempView('df_v')

spark.sql("select * from df_v").show()

+---+
| id|
+---+
|  0|
+---+



In [40]:
spark.sql("""select current_date() date, 
           current_timestamp() time_stamp
           """).show( truncate=False)

+----------+--------------------------+
|date      |time_stamp                |
+----------+--------------------------+
|2025-12-05|2025-12-05 10:47:51.880017|
+----------+--------------------------+



In [41]:
spark.sql("""select current_date() date,
                  year(current_date()) year,
                   quarter(current_date()) quarter,
                    month (current_date()) month  
                    
                    """).show()

+----------+----+-------+-----+
|      date|year|quarter|month|
+----------+----+-------+-----+
|2025-12-05|2025|      4|   12|
+----------+----+-------+-----+



In [42]:
spark.sql("""
    SELECT date_format(current_date(), 'dd-MM-yyyy') AS formatted_date
""").show()


+--------------+
|formatted_date|
+--------------+
|    05-12-2025|
+--------------+



In [63]:
emp_df.show()

+------+-----+-------+-------+-------+
|emp_id| name|   dept| salary|dept_id|
+------+-----+-------+-------+-------+
|     1| Ravi|  Sales|50000.0|     10|
|     2|Priya|     HR|60000.0|     20|
|     3| John|  Sales|45000.0|     10|
|     4|Meera|Finance|70000.0|     30|
|     5|Rahul|   NULL|55000.0|   NULL|
+------+-----+-------+-------+-------+



In [43]:
spark.sql("""
          select emp_id,name,salary, 
          rank() over(order by salary desc) as emp_rank,
          dense_rank() over(order by salary desc) as emp_dense_rank,
          row_number() over(order by salary desc) as row_number,
          lag(salary) over(order by salary desc) as lag_sal,
          lead(salary) over(order by salary desc) as lead_sal,
          first_value(salary) over(order by salary desc) as first_sal,
          last_value(salary) over(order by salary desc ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) as last_sal,
          sum(salary) over(order by salary desc ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) as sum_sal,
          sum(salary) over(order by salary desc ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) as running_sum
          
          from emp
          """).show()
            

+------+-----+-------+--------+--------------+----------+-------+--------+---------+--------+--------+-----------+
|emp_id| name| salary|emp_rank|emp_dense_rank|row_number|lag_sal|lead_sal|first_sal|last_sal| sum_sal|running_sum|
+------+-----+-------+--------+--------------+----------+-------+--------+---------+--------+--------+-----------+
|     4|Meera|70000.0|       1|             1|         1|   NULL| 60000.0|  70000.0| 45000.0|280000.0|    70000.0|
|     2|Priya|60000.0|       2|             2|         2|70000.0| 55000.0|  70000.0| 45000.0|280000.0|   130000.0|
|     5|Rahul|55000.0|       3|             3|         3|60000.0| 50000.0|  70000.0| 45000.0|280000.0|   185000.0|
|     1| Ravi|50000.0|       4|             4|         4|55000.0| 45000.0|  70000.0| 45000.0|280000.0|   235000.0|
|     3| John|45000.0|       5|             5|         5|50000.0|    NULL|  70000.0| 45000.0|280000.0|   280000.0|
+------+-----+-------+--------+--------------+----------+-------+--------+------

In [46]:
spark.sql("""
      select emp_id,name,salary,dept, rank() over(partition by dept order by salary desc ) rank_sal_dept
     from  emp""").show()

+------+-----+-------+-------+-------------+
|emp_id| name| salary|   dept|rank_sal_dept|
+------+-----+-------+-------+-------------+
|     5|Rahul|55000.0|   NULL|            1|
|     4|Meera|70000.0|Finance|            1|
|     2|Priya|60000.0|     HR|            1|
|     1| Ravi|50000.0|  Sales|            1|
|     3| John|45000.0|  Sales|            2|
+------+-----+-------+-------+-------------+



In [48]:
spark.sql("""
         with emp_sal as(
          select emp_id,name,salary, 
          rank() over(order by salary asc) as emp_rank from emp)
          select * from emp_sal where emp_rank <3
          """).show()

+------+----+-------+--------+
|emp_id|name| salary|emp_rank|
+------+----+-------+--------+
|     3|John|45000.0|       1|
|     1|Ravi|50000.0|       2|
+------+----+-------+--------+



In [53]:
emp_data = [
    (1, "Ravi",   "Sales",   50000.0, 10),
    (1, "Ravi",   "Sales",   60000.0, 10),
    (1, "Ravi",   "Sales",   70000.0, 10),
    (2, "Priya",  "HR",      60000.0, 20),
    (2, "Priya",  "HR",      70000.0, 20),
    (3, "John",   "Sales",   45000.0, 10),
    (4, "Meera",  "Finance", 70000.0, 30),
    (5, "Rahul",  None,      55000.0, None),
    (5, "Rahul",  None,      65000.0, None)
]

emp_schema = StructType([
    StructField("emp_id", IntegerType(), False),
    StructField("name",   StringType(),  True),
    StructField("dept",   StringType(),  True),
    StructField("salary", DoubleType(),  True),
    StructField("dept_id",IntegerType(), True)
])

emp_df1 = spark.createDataFrame(emp_data, schema=emp_schema)
emp_df1.show()


+------+-----+-------+-------+-------+
|emp_id| name|   dept| salary|dept_id|
+------+-----+-------+-------+-------+
|     1| Ravi|  Sales|50000.0|     10|
|     1| Ravi|  Sales|60000.0|     10|
|     1| Ravi|  Sales|70000.0|     10|
|     2|Priya|     HR|60000.0|     20|
|     2|Priya|     HR|70000.0|     20|
|     3| John|  Sales|45000.0|     10|
|     4|Meera|Finance|70000.0|     30|
|     5|Rahul|   NULL|55000.0|   NULL|
|     5|Rahul|   NULL|65000.0|   NULL|
+------+-----+-------+-------+-------+



In [54]:
emp_df1.createOrReplaceTempView("emp1")

In [55]:
spark.sql("""
          select distinct * from emp1""").show()

+------+-----+-------+-------+-------+
|emp_id| name|   dept| salary|dept_id|
+------+-----+-------+-------+-------+
|     1| Ravi|  Sales|50000.0|     10|
|     1| Ravi|  Sales|60000.0|     10|
|     2|Priya|     HR|60000.0|     20|
|     2|Priya|     HR|70000.0|     20|
|     3| John|  Sales|45000.0|     10|
|     4|Meera|Finance|70000.0|     30|
|     1| Ravi|  Sales|70000.0|     10|
|     5|Rahul|   NULL|55000.0|   NULL|
|     5|Rahul|   NULL|65000.0|   NULL|
+------+-----+-------+-------+-------+



In [62]:
spark.sql("""
          with emp_rank_data as (
           select emp_id,name,dept,salary,dept_id , rank() over( partition by emp_id order by salary desc) as emp_rank from emp1)
           select emp_id,name,dept,salary,dept_id from emp_rank_data where emp_rank =1
           """).show()

+------+-----+-------+-------+-------+
|emp_id| name|   dept| salary|dept_id|
+------+-----+-------+-------+-------+
|     1| Ravi|  Sales|70000.0|     10|
|     2|Priya|     HR|70000.0|     20|
|     3| John|  Sales|45000.0|     10|
|     4|Meera|Finance|70000.0|     30|
|     5|Rahul|   NULL|65000.0|   NULL|
+------+-----+-------+-------+-------+



In [71]:
emp_df.createOrReplaceGlobalTempView("emp_g")

In [72]:
spark.sql("select * from global_temp.emp_g").show()

+------+-----+-------+-------+-------+
|emp_id| name|   dept| salary|dept_id|
+------+-----+-------+-------+-------+
|     1| Ravi|  Sales|50000.0|     10|
|     2|Priya|     HR|60000.0|     20|
|     3| John|  Sales|45000.0|     10|
|     4|Meera|Finance|70000.0|     30|
|     5|Rahul|   NULL|55000.0|   NULL|
+------+-----+-------+-------+-------+



In [73]:
spark.catalog.listTables()

[Table(name='dept', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='df_v', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='emp', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='emp1', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True)]

In [69]:
spark.catalog.currentCatalog()

'spark_catalog'

In [70]:
spark.catalog.currentDatabase()

'default'

In [74]:
spark.sql("select * from global_temp.emp_g").show()

+------+-----+-------+-------+-------+
|emp_id| name|   dept| salary|dept_id|
+------+-----+-------+-------+-------+
|     1| Ravi|  Sales|50000.0|     10|
|     2|Priya|     HR|60000.0|     20|
|     3| John|  Sales|45000.0|     10|
|     4|Meera|Finance|70000.0|     30|
|     5|Rahul|   NULL|55000.0|   NULL|
+------+-----+-------+-------+-------+

